In [31]:
import operator
import os
import sys
import re
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
from utils.env_util import *
from tools.search import bocha_websearch_tool
from langgraph_utils.common_util import gen_mermaid
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import Annotated, Literal
from langgraph.types import Send
from pydantic import BaseModel, Field
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL
from langgraph.graph import MessagesState, END
from langgraph.types import Command
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent

## 构造 code agnet node 和 research agent node

In [32]:
class State(MessagesState):
    next: str
    
llm = ChatOpenAI(
    openai_api_key=get_openai_api_key(),
    # model_name=get_default_model(),
    model_name="deepseek-ai/DeepSeek-V3",
    base_url=get_openai_base_url(),
    temperature=0.0,
)

repl = PythonREPL()

@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code and do math. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n\`\`\`python\n{code}\n\`\`\`\nStdout: {result}"
    return result_str


# NOTE: 这将执行任意代码，如果没有进行沙盒处理，这可能是不安全的
code_agent = create_react_agent(llm, tools=[python_repl_tool])


def code_node(state: State) -> Command[Literal["supervisor"]]:
    result = code_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="coder")
            ]
        },
        goto="supervisor",
    )

#######################################################################

research_agent = create_react_agent(
    llm, tools=[bocha_websearch_tool], prompt="You are a researcher. DO NOT do any math calculation."
)

def research_node(state: State) -> Command[Literal["supervisor"]]:
    result = research_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="researcher")
            ]
        },
        goto="supervisor",
    )

🙈 OPENAI_API_KEY: sk-hybehtt*******************************lpkkvcvojw
👀 OPENAI_BASE_URL: https://api.siliconflow.cn/v1


## 创建监督者 Agent

In [33]:
members = ["researcher", "coder"]
# 监督者是LLM节点，它选择下一个agent来处理和控制何时终止
options = members + ["FINISH"]

system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    f" following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)


class Router(TypedDict):
    """Worker to route to next. If no workers needed, route to FINISH."""
    next: Literal[*options]


def supervisor_node(state: State) -> Command[Literal[*members, "__end__"]]:
    messages = [
        {"role": "system", "content": system_prompt},
    ] + state["messages"]
    response = llm.with_structured_output(Router).invoke(messages)
    goto = response["next"]
    if goto == "FINISH":
        goto = END
    return Command(goto=goto, update={"next": goto})

## 构建图

In [34]:
builder = StateGraph(State)
builder.add_edge(START, "supervisor")
builder.add_node("supervisor", supervisor_node)
builder.add_node("researcher", research_node)
builder.add_node("coder", code_node)
graph = builder.compile()

gen_mermaid(graph=graph, file_name="multi_agent_supervisor.mmd")
graph.get_graph().print_ascii()

✏️ 已生成 mermaid 文件 /workspace/Agent/langgraph_demo/resources/multi_agent_supervisor.mmd
                    +-----------+                     
                    | __start__ |                     
                    +-----------+                     
                           *                          
                           *                          
                           *                          
                    +------------+                    
                    | supervisor |                    
                   .+------------+.                   
               ....        .       ....               
            ...            .           ...            
          ..               .              ..          
+------------+         +-------+         +---------+  
| researcher |         | coder |         | __end__ |  
+------------+         +-------+         +---------+  


## 调用多 Agent 系统

In [35]:
for s in graph.stream(
    {
        "messages": [
            (
                "user",
                "北京到上海的距离",
            )
        ]
    },
    subgraphs=True,
):
    print(s)
    print("----")

((), {'supervisor': {'next': 'researcher'}})
----
(('researcher:944ad962-dc99-429e-245f-faa9ece3e1e2',), {'agent': {'messages': [AIMessage(content='要回答“北京到上海的距离”这个问题，我需要先搜索相关的信息。由于这是一个常见的地理问题，可以通过网络搜索获取准确的数据。我将使用网页搜索工具来查找北京到上海的直线距离或实际交通距离。\n\n', additional_kwargs={'tool_calls': [{'id': '01966b0eacbac94ad3c4d8db0d6d2b0a', 'function': {'arguments': '{"query":"北京到上海的距离","count":1}', 'name': 'bocha_websearch_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 256, 'total_tokens': 332, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '01966b0e8b7756e33a7a829fce61d2dc', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-066235a8-a8fa-4774-92e8-4ec8643a0736-0', tool_calls=[{'name': 'bocha_websearch_tool', 'args': 

In [36]:
for s in graph.stream(
    {"messages": [("user", "What's the answer of 1+4?")]}, subgraphs=True
):
    print(s)
    print("*" * 80)

((), {'supervisor': {'next': 'coder'}})
********************************************************************************
(('coder:6f46a5ef-5266-eadf-14ae-503627ff1c7b',), {'agent': {'messages': [AIMessage(content="To find the answer to 1 + 4, I can use simple arithmetic. The sum of 1 and 4 is straightforward, but I'll verify it using the Python REPL tool to ensure accuracy.", additional_kwargs={'tool_calls': [{'id': '01966b0f12065d41d3b5ae304ab2fb70', 'function': {'arguments': '{"code":"print(1 + 4)"}', 'name': 'python_repl_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 211, 'total_tokens': 281, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '01966b0ef1136b6c3b53aadd8ab7e933', 'finish_reason': 'tool_calls',